### Removing Duplicates

In [2]:
import numpy as np
import pandas as pd

In [4]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                     'k2': [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [6]:
# whether each row is duplicated(has been observed in a previous row) or not
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [9]:
# drop_duplicates returns a DataFrame where the duplicated
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


In [11]:
# both of these methods by default consider all of the columns
# we can specify particular columns
data['v1'] = range(7)
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [12]:
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
1,two,1,1


In [13]:
# duplicated and drop_duplicates by default keep the first observed value 
# passing keep='last' will return the last one
data.drop_duplicates(['k1','k2'], keep = 'last')

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


### Transforming Data Using a Function or Mapping

In [14]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
   ....:                               'Pastrami', 'corned beef', 'Bacon',
   ....:                               'pastrami', 'honey ham', 'nova lox'],
   ....:                      'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [16]:
# add a column indicating the type of animal that each food came from
# let's write down a mapping of each distinct meat type to the kind of animal
meat_to_animal = {
  'bacon': 'pig',
  'pulled pork': 'pig',
  'pastrami': 'cow',
  'corned beef': 'cow',
  'honey ham': 'pig',
  'nova lox': 'salmon'
}

In [17]:
lowercased = data['food'].str.lower()
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [18]:
data['animal'] = lowercased.map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [25]:
# we could also passed a function that does all the work:
data['food'].map(lambda x:meat_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

## Replacing Values

In [26]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [27]:
# the -999 values might be sentinel values for missing data.
# To replace these with NA values that pandas understands, we can use replace
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [28]:
data.replace([-999,-1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [29]:
data.replace([-999,-1000], [np.nan,0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [30]:
# the argument passed can also be a dict
data.replace({-999:np.nan, -1000:0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

### Renaming Axis Indexes

In [33]:
# axis labels can be similarly transformed by a function or mapping
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                     index=['Ohio', 'Colorado', 'New York'],
                     columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [34]:
transform = lambda x:x[:4].upper()
data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

In [35]:
# you can assign to index, modifying the DataFrame in-place
data.index = data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [36]:
# If you want to create a transformed version of a dataset
data.rename(index=str.title,columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


In [44]:
# inplace
data.rename(index={'OHIO':'INDIANA'}, inplace=True)
data

,one,two,three,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


### Discretization and Binning

In [45]:
# Continuous data is often discretized or otherwise separated into “bins” for analysis
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]
bins = [18,25,35,60,100]
cats = pd.cut(ages, bins)
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [46]:
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [49]:
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]],
              closed='right',
              dtype='interval[int64]')

In [50]:
pd.value_counts(cats)

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

In [51]:
# change which side is closed by passing right=False
pd.cut(ages,[18,26,36,61,100],right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

In [52]:
# you can also pass your won bin names
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels = group_names)

[Youth, Youth, Youth, YoungAdult, Youth, ..., YoungAdult, Senior, MiddleAged, MiddleAged, YoungAdult]
Length: 12
Categories (4, object): [Youth < YoungAdult < MiddleAged < Senior]

In [53]:
# if you pass a integer number of bins to cut instead of explicit bin edges
# it will compute equal-length bins based on the minimum and maximum values
data = np.random.rand(20)
pd.cut(data, 4, precision=2)

[(0.12, 0.34], (0.56, 0.78], (0.34, 0.56], (0.34, 0.56], (0.78, 1.0], ..., (0.12, 0.34], (0.12, 0.34], (0.12, 0.34], (0.56, 0.78], (0.56, 0.78]]
Length: 20
Categories (4, interval[float64]): [(0.12, 0.34] < (0.34, 0.56] < (0.56, 0.78] < (0.78, 1.0]]

In [54]:
data = np.random.randn(1000)  # Normally distributed
cats = pd.qcut(data,4) #cut into quartiles
cats

[(0.0461, 0.678], (-0.652, 0.0461], (0.678, 2.871], (-2.977, -0.652], (-0.652, 0.0461], ..., (0.678, 2.871], (0.0461, 0.678], (-0.652, 0.0461], (0.678, 2.871], (-2.977, -0.652]]
Length: 1000
Categories (4, interval[float64]): [(-2.977, -0.652] < (-0.652, 0.0461] < (0.0461, 0.678] < (0.678, 2.871]]

In [55]:
pd.value_counts(cats)

(0.678, 2.871]      250
(0.0461, 0.678]     250
(-0.652, 0.0461]    250
(-2.977, -0.652]    250
dtype: int64

In [56]:
# you can pass your own quantiles(numbers between 0 and 1)
cats = pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.])
cats

[(0.0461, 1.288], (-1.187, 0.0461], (0.0461, 1.288], (-2.977, -1.187], (-1.187, 0.0461], ..., (0.0461, 1.288], (0.0461, 1.288], (-1.187, 0.0461], (1.288, 2.871], (-1.187, 0.0461]]
Length: 1000
Categories (4, interval[float64]): [(-2.977, -1.187] < (-1.187, 0.0461] < (0.0461, 1.288] < (1.288, 2.871]]

In [57]:
cats.value_counts()

(-2.977, -1.187]    100
(-1.187, 0.0461]    400
(0.0461, 1.288]     400
(1.288, 2.871]      100
dtype: int64

### Detecting and Filtering Outliers

In [59]:
data = pd.DataFrame(np.random.randn(1000, 4)) # normally distributed data
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.036060,-0.030323,0.031688,-0.047465
std,0.981425,1.029813,1.023255,0.982296
min,-3.239115,-3.315667,-3.486774,-2.598375
25%,-0.605372,-0.725314,-0.661214,-0.728507
50%,0.040320,-0.021729,0.059276,-0.029687
75%,0.713699,0.642806,0.745597,0.577378
max,2.850934,3.371107,3.496317,3.291721


In [61]:
# suppose you want to find values in one of the columns exceeding 3 in absolute value
col = data[2]
col[np.abs(col)>3]

454    3.304249
480   -3.486774
794    3.496317
Name: 2, dtype: float64

In [65]:
# To select all rows having a value exceeding 3 or -3, you can use any method
data[(np.abs(data)>3).any(1)]

,0,1,2,3
40,0.748267,-3.315667,0.087250,-0.229458
135,1.965049,-3.067702,-0.551732,-0.138083
263,0.137499,3.371107,-1.517419,-1.532701
350,-1.861298,3.228854,0.435324,-0.684603
454,-0.061133,-1.066740,3.304249,-0.543922
480,-0.220469,0.488661,-3.486774,-0.186311
675,-3.239115,1.856846,-0.070898,0.107146
760,0.318601,-3.191195,0.143973,-1.060706
794,-0.596381,-0.130802,3.496317,-0.546606
886,-0.762085,-0.058812,-0.094650,3.291721


In [74]:
(np.abs(data)>3).any(0)

0    True
1    True
2    True
3    True
dtype: bool

In [76]:
(np.abs(data)>3).any(1)[:5]

0    False
1    False
2    False
3    False
4    False
dtype: bool

In [80]:
# cap values outside the interval -3 to 3
# e.g. -4 and 4 will set to -3 and 3
data[np.abs(data)>3] = np.sign(data)*3
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.036299,-0.030349,0.031374,-0.047757
std,0.980655,1.026221,1.019197,0.981347
min,-3.000000,-3.000000,-3.000000,-2.598375
25%,-0.605372,-0.725314,-0.661214,-0.728507
50%,0.040320,-0.021729,0.059276,-0.029687
75%,0.713699,0.642806,0.745597,0.577378
max,2.850934,3.000000,3.000000,3.000000


In [82]:
np.sign(data).head()

,0,1,2,3
0,-1.0,1.0,1.0,-1.0
1,1.0,1.0,-1.0,1.0
2,1.0,1.0,1.0,-1.0
3,-1.0,1.0,-1.0,1.0
4,-1.0,-1.0,-1.0,1.0


### Permutation and Random Sampling(randomly reordering)

In [83]:
df = pd.DataFrame(np.arange(5 * 4).reshape((5, 4)))
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [85]:
sampler = np.random.permutation(5)
sampler

array([3, 2, 1, 0, 4])

In [86]:
df.take(sampler)

,0,1,2,3
3,12,13,14,15
2,8,9,10,11
1,4,5,6,7
0,0,1,2,3
4,16,17,18,19


In [95]:
df.loc[sampler.tolist(),:]

,0,1,2,3
3,12,13,14,15
2,8,9,10,11
1,4,5,6,7
0,0,1,2,3
4,16,17,18,19


In [96]:
df.iloc[sampler.tolist()]

,0,1,2,3
3,12,13,14,15
2,8,9,10,11
1,4,5,6,7
0,0,1,2,3
4,16,17,18,19


In [99]:
df.iloc[index:sampler.tolist()]

NameError: name 'index' is not defined

In [100]:
# To select a random subset without replacement, you can use the sample method
df.sample(n=3)

,0,1,2,3
2,8,9,10,11
4,16,17,18,19
0,0,1,2,3


In [102]:
# To generate a sample with replacement, pass replace=True
choices = pd.Series([5,7,-1,6,4])
draws = choices.sample(n=10, replace=True)
draws

0    5
4    4
1    7
1    7
1    7
0    5
2   -1
4    4
3    6
1    7
dtype: int64

## Computing Indicator/Dummy Variables